In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, DateType, FloatType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Step 1: Data Extraction
    logger.info("Extracting data from flat file source.")
    flat_file_df = spark.read.option("header", "true").csv("/Volumes/genai_demo/ssis/ssis/SampleCurrencyData.txt")
    
    logger.info("Extracting data from DimCurrency and DimDate tables.")
    dim_currency_df = spark.table("sqlserver_catalog.dbo.dimcurrency1")
    dim_date_df = spark.table("sqlserver_catalog.dbo.dimdate1")

    # Step 2: Data Transformation
    # Lookup Currency Key
    logger.info("Performing lookup for Currency Key.")
    enriched_currency_df = flat_file_df.join(
        dim_currency_df,
        flat_file_df.CurrencyID == dim_currency_df.CurrencyAlternateKey,
        "inner"
    ).select(
        flat_file_df["*"],
        dim_currency_df["CurrencyKey"]
    )

    # Lookup Date Key
    logger.info("Performing lookup for Date Key.")
    enriched_date_df = enriched_currency_df.join(
        dim_date_df,
        enriched_currency_df.CurrencyDate == dim_date_df.FullDateAlternateKey,
        "inner"
    ).select(
        enriched_currency_df["*"],
        dim_date_df["DateKey"]
    )

    # Step 3: Data Loading
    logger.info("Loading transformed data into Unity Catalog table.")
    spark.sql("DROP TABLE IF EXISTS sqlserver_catalog.dbo.ssisresult")
    enriched_date_df.write.format("delta").mode("overwrite").saveAsTable("sqlserver_catalog.dbo.ssisresult")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process: %s", str(e))
    raise
